In [9]:
import pdfplumber

def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

text_ai = extract_text_from_pdf("10033-abit.pdf")
text_ai_product=extract_text_from_pdf("10130-abit.pdf")
print(text_ai[:1000])  

Учебный план
ОП Искусственный интеллект
атратс
ыртсемеС Наименование модулей, дисциплин, практики и аттестации
де.з
в
ьтсокмеодурТ
.сач
в
ьтсокмеодурТ
Блок 1. Модули (дисциплины) 60 2160
Индивидуальная профессиональная подготовка (по профессиональным областям, по
48 1728
профессиональным ролям, по уровню сложности и др.)
Обязательные дисциплины. 1 семестр 3 108
1 Воркшоп по созданию продукта на данных / Data Product Development Workshop 3 108
Пул выборных дисциплин. 1 семестр 15 540
1 Практика применения машинного обучения 6 216
1 Алгоритмы и структуры данных 3 108
1 Математическая статистика 3 108
1 Разработка веб-приложений (Python Backend) 6 216
1 Программирование на С++ 3 108
1 Введение в МО (Python) и Продвинутое МО (Python) 3 108
1 Технологии обработки естественного языка 6 216
1 Автоматическое машинное обучение 3 108
1 Обработка и генерация изображений 3 108
1 Проектирование и разработка рекомендательных систем (продвинутый уровень) 6 216
1 Основы глубокого обучения 3 108
1 Прод

In [14]:
import re
import pandas as pd

def parse_study_plan_v2(text):
    lines = text.splitlines()
    lines = [line.strip() for line in lines if line.strip() != '']

    current_block = None
    data = []

    block_pattern = re.compile(r'Блок\s+(\d+)')
    sem_numbers_pattern = re.compile(r'[\d, ]+')  

    for line in lines:
        block_match = block_pattern.search(line)
        if block_match:
            current_block = f"Блок {block_match.group(1)}"
            continue

        hours_load_match = re.search(r'(\d+)\s+(\d+)$', line)
        if not hours_load_match:
            continue

        hours = int(hours_load_match.group(1))
        load = int(hours_load_match.group(2))

        line_wo_hours = line[:hours_load_match.start()].strip()


        sem_match = sem_numbers_pattern.match(line_wo_hours)
        if sem_match:
            sem_str = sem_match.group(0).strip(', ').replace(' ', '')
            semesters = sem_str.split(',') if sem_str else []
            name = line_wo_hours[sem_match.end():].strip()
        else:
            semesters = []
            name = line_wo_hours

        if not semesters:
            semesters = [None]

        for sem in semesters:
            data.append({
                "block": current_block,
                "semester": int(sem) if sem and sem.isdigit() else None,
                "name": name,
                "hours": hours,
                "load": load
            })

    df = pd.DataFrame(data)
    return df

df_ai = parse_study_plan_v2(text_ai)
df_ai_product= parse_study_plan_v2(text_ai_product)

In [51]:
df_ai.to_csv('ai_plan.csv',index=False)
df_ai_product.to_csv('ai_product_plan.csv',index=False)

In [26]:
from stop_words import get_stop_words
russian_stopwords = set(get_stop_words('russian'))

In [30]:
import requests
from bs4 import BeautifulSoup

def parse_program_page(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    for script in soup(["script", "style", "noscript"]):
        script.decompose()

    content = {}

    for header_tag in ['h1', 'h2', 'h3', 'h4', 'h5']:
        headers = soup.find_all(header_tag)
        for h in headers:
            title = h.get_text(strip=True)
            texts = []
            sibling = h.find_next_sibling()
            while sibling and sibling.name not in ['h1', 'h2', 'h3', 'h4', 'h5']:
                # Собираем текст со всех вложенных элементов с пробелами
                texts.append(" ".join(sibling.stripped_strings))
                sibling = sibling.find_next_sibling()
            if texts:
                content[title] = "\n".join(texts)

    return content

url = "https://abit.itmo.ru/program/master/ai"
program_info = parse_program_page(url)

for section, text in program_info.items():
    print(f"== {section} ==\n{text}\n")


== Искусственный интеллект ==
институт прикладных компьютерных наук
форма обучения очная длительность 2 года язык обучения русский стоимость контрактного обучения (год) 599 000 ₽ общежитие да военный учебный центр да гос. аккредитация да дополнительные возможности Онлайн, Трек аспирантуры, ПИШ, Программа в сфере ИИ
Менеджер программы Елизавета Витальевна Василенко aitalents@itmo.ru +7 (999) 526-79-88 Программа в соцсетях ВКонтакте Сайт Telegram
Даты вступительного экзамена

== направления подготовки ==
09.04.01 Информатика и вычислительная техника 51 бюджетных 4 целевая 55 контрактных 11.04.02 Инфокоммуникационные технологии и системы связи 80 бюджетных 5 целевая 25 контрактных 27.04.05 Инноватика 80 бюджетных 5 целевая 40 контрактных

== о программе ==
Создавайте AI-продукты и технологии, которые меняют мир.
Основа обучения на программе – проектный подход. Магистранты работают над проектами ведущих компаний — X5 Group, Ozon Банк, МТС, Sber AI, Норникель, Napoleon IT, Genotek, Raft, AI

In [33]:
url_ai_product = "https://abit.itmo.ru/program/master/ai_product"
url_ai = "https://abit.itmo.ru/program/master/ai"
program_info_ai = parse_program_page(url_ai)
program_info_ai_product = parse_program_page(url_ai_product)

In [50]:
import asyncio
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from aiogram import Bot, Dispatcher, types
from aiogram.filters import Command
from aiogram.fsm.state import StatesGroup, State
from aiogram.fsm.context import FSMContext
import nest_asyncio
from stop_words import get_stop_words

def parse_program_page(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    for script in soup(["script", "style", "noscript"]):
        script.decompose()
    content = {}
    for header_tag in ['h1', 'h2', 'h3', 'h4', 'h5']:
        headers = soup.find_all(header_tag)
        for h in headers:
            title = h.get_text(strip=True)
            texts = []
            sibling = h.find_next_sibling()
            while sibling and sibling.name not in ['h1','h2','h3','h4','h5']:
                texts.append(" ".join(sibling.stripped_strings))
                sibling = sibling.find_next_sibling()
            if texts:
                content[title] = " ".join(texts)
    return content

url_ai = "https://abit.itmo.ru/program/master/ai"
url_ai_product = "https://abit.itmo.ru/program/master/ai_product"

info_ai = parse_program_page(url_ai)
info_ai_product = parse_program_page(url_ai_product)

# df_ai = pd.read_csv('ai_plan.csv')
# df_ai_product = pd.read_csv('ai_product_plan.csv')

texts = []
labels = []

for i, row in df_ai.iterrows():
    desc = f"{row.get('name', '')}, семестр {row.get('semester', '')}, трудоемкость {row.get('hours', '')} з.ед., {row.get('load', '')} часов"
    texts.append(desc)
    labels.append("Учебный план AI")

for i, row in df_ai_product.iterrows():
    desc = f"{row.get('name', '')}, семестр {row.get('semester', '')}, трудоемкость {row.get('hours', '')} з.ед., {row.get('load', '')} часов"
    texts.append(desc)
    labels.append("Учебный план AI Product")

for section, text in info_ai.items():
    texts.append(text)
    labels.append(f"Информация AI - {section}")

for section, text in info_ai_product.items():
    texts.append(text)
    labels.append(f"Информация AI Product - {section}")

russian_stopwords = get_stop_words('russian')
vectorizer = TfidfVectorizer(stop_words=russian_stopwords)
tfidf_matrix = vectorizer.fit_transform(texts)

class RecommendStates(StatesGroup):
    waiting_for_background = State()
    waiting_for_interests = State()
    waiting_for_goals = State()

def get_elective_courses():
    electives_ai = df_ai[df_ai.get('type', '') == 'elective'] if 'type' in df_ai.columns else pd.DataFrame()
    electives_ai_product = df_ai_product[df_ai_product.get('type', '') == 'elective'] if 'type' in df_ai_product.columns else pd.DataFrame()
    electives = pd.concat([electives_ai, electives_ai_product], ignore_index=True)
    elective_names = []
    elective_texts = []
    for _, row in electives.iterrows():
        desc = f"{row.get('name', '')}, семестр {row.get('semester', '')}, трудоемкость {row.get('hours', '')} з.ед., {row.get('load', '')} часов"
        elective_names.append(row.get('name', ''))
        elective_texts.append(desc)
    return elective_names, elective_texts

if 'type' in df_ai.columns and 'type' in df_ai_product.columns:
    elective_names, elective_texts = get_elective_courses()
else:
    elective_names = []
    elective_texts = []
    for _, row in pd.concat([df_ai, df_ai_product], ignore_index=True).iterrows():
        desc = f"{row.get('name', '')}, семестр {row.get('semester', '')}, трудоемкость {row.get('hours', '')} з.ед., {row.get('load', '')} часов"
        elective_names.append(row.get('name', ''))
        elective_texts.append(desc)

elective_vectorizer = TfidfVectorizer(stop_words=russian_stopwords)
elective_matrix = elective_vectorizer.fit_transform(elective_texts)

API_TOKEN = ""

bot = Bot(token=API_TOKEN)
dp = Dispatcher()

user_mode = {}
user_program = {}

@dp.message(Command("start"))
async def start_handler(message: types.Message):
    user_mode[message.from_user.id] = "choose_program"
    await message.answer(
        "Привет! Выберите программу:\n"
        "1 — Искусственный интеллект\n"
        "2 — Управление ИИ-продуктами\n"
        "Если не знаете — напишите 'не знаю'\n\n"
        "После выбора вы можете:\n"
        "/plan — узнать учебный план\n"
        "/info — получить общую информацию\n"
        "/recommend — получить рекомендации по предметам"
    )

@dp.message(Command("plan"))
async def plan_command(message: types.Message):
    uid = message.from_user.id
    if uid not in user_program or user_program[uid] is None:
        await message.answer("Пожалуйста, сначала выберите программу командой /start и выберите 1 или 2.")
        return
    user_mode[uid] = "plan_mode"
    await message.answer(
        "Введите номер семестра, чтобы узнать предметы (например, '1' или '2'), "
        "или напишите 'все', чтобы увидеть полный учебный план."
    )

@dp.message(Command("info"))
async def info_command(message: types.Message):
    uid = message.from_user.id
    if uid not in user_program or user_program[uid] is None:
        await message.answer("Пожалуйста, сначала выберите программу командой /start и выберите 1 или 2.")
        return
    user_mode[uid] = "info_mode"
    await message.answer("Задайте вопрос по общей информации программы.")

@dp.message(Command("recommend"))
async def recommend_start(message: types.Message, state: FSMContext):
    uid = message.from_user.id
    if uid not in user_program or user_program[uid] is None:
        await message.answer("Пожалуйста, сначала выберите программу командой /start и выберите 1 или 2.")
        return
    await state.set_state(RecommendStates.waiting_for_background)
    await message.answer("Расскажите, пожалуйста, о вашем образовании или бэкграунде (например, математика, программирование, экономика и т.п.).")

@dp.message(RecommendStates.waiting_for_background)
async def recommend_background(message: types.Message, state: FSMContext):
    await state.update_data(background=message.text.lower())
    await state.set_state(RecommendStates.waiting_for_interests)
    await message.answer("Какие у вас интересы? Например, ИИ, управление, аналитика, программирование и т.д.")

@dp.message(RecommendStates.waiting_for_interests)
async def recommend_interests(message: types.Message, state: FSMContext):
    await state.update_data(interests=message.text.lower())
    await state.set_state(RecommendStates.waiting_for_goals)
    await message.answer("Какие у вас цели после обучения? Например, разработка ПО, исследовательская работа, управление продуктом.")

@dp.message(RecommendStates.waiting_for_goals)
async def recommend_goals(message: types.Message, state: FSMContext):
    await state.update_data(goals=message.text.lower())
    data = await state.get_data()

    query = f"{data['background']} {data['interests']} {data['goals']}"
    query_vec = elective_vectorizer.transform([query])
    scores = cosine_similarity(query_vec, elective_matrix).flatten()
    top_indices = scores.argsort()[::-1][:7]
    recommendations = [elective_names[i] for i in top_indices if scores[i] > 0.1]

    if not recommendations:
        await message.answer("К сожалению, не смог подобрать подходящие выборные дисциплины по вашему описанию. Попробуйте уточнить ввод.")
    else:
        response = "Рекомендую обратить внимание на следующие выборные дисциплины:\n"
        for rec in recommendations:
            response += f"- {rec}\n"
        await message.answer(response)

    await state.clear()

@dp.message()
async def message_handler(message: types.Message):
    uid = message.from_user.id
    text = message.text.lower().strip()

    if user_mode.get(uid) == "choose_program":
        if "1" in text or "искусственный" in text or "ai" in text:
            user_program[uid] = "AI"
            user_mode[uid] = None
            await message.answer(
                "Вы выбрали Искусственный интеллект.\n"
                "Используйте /plan для учебного плана или /info для общей информации."
            )
        elif "2" in text or "управление" in text or "продукт" in text:
            user_program[uid] = "AI Product"
            user_mode[uid] = None
            await message.answer(
                "Вы выбрали Управление ИИ-продуктами.\n"
                "Используйте /plan для учебного плана или /info для общей информации."
            )
        elif "не знаю" in text:
            user_program[uid] = None
            user_mode[uid] = None
            await message.answer(
                "Вы не выбрали конкретную программу.\n"
                "Используйте /plan для учебного плана обеих программ или /info для общей информации."
            )
        else:
            await message.answer("Пожалуйста, выберите программу, написав 1, 2 или 'не знаю'.")

    elif user_mode.get(uid) == "plan_mode":
        df = None
        if user_program.get(uid) == "AI":
            df = df_ai
        elif user_program.get(uid) == "AI Product":
            df = df_ai_product
        else:
            df = pd.concat([df_ai, df_ai_product], ignore_index=True)

        if text == "все":
            response = ""
            semesters = sorted(df['semester'].dropna().unique())
            for sem in semesters:
                response += f"Семестр {int(sem)}:\n"
                df_sem = df[df['semester'] == sem]
                for _, row in df_sem.iterrows():
                    response += f"- {row['name']} ({row['hours']} з.ед., {row['load']} часов)\n"
                response += "\n"
            for part in split_message(response):
                await message.answer(part)
        else:
            sem_nums = [int(s) for s in re.findall(r'\d+', text)]
            if sem_nums:
                sem = sem_nums[0]
                df_sem = df[df['semester'] == sem]
                if df_sem.empty:
                    await message.answer(f"Данные по семестру {sem} не найдены.")
                else:
                    response = f"Предметы семестра {sem}:\n"
                    for _, row in df_sem.iterrows():
                        response += f"- {row['name']} ({row['hours']} з.ед., {row['load']} часов)\n"
                    for part in split_message(response):
                        await message.answer(part)
            else:
                await message.answer("Пожалуйста, введите номер семестра (например, '1' или '2') или 'все'.")

    elif user_mode.get(uid) == "info_mode":
        query_vec = vectorizer.transform([text])
        scores = cosine_similarity(query_vec, tfidf_matrix).flatten()

        if user_program[uid] == "AI":
            indices = [i for i, label in enumerate(labels) if label.startswith("Информация AI")]
        elif user_program[uid] == "AI Product":
            indices = [i for i, label in enumerate(labels) if label.startswith("Информация AI Product")]
        else:
            indices = [i for i, label in enumerate(labels) if label.startswith("Информация AI") or label.startswith("Информация AI Product")]

        filtered_scores = [(i, scores[i]) for i in indices]
        filtered_scores.sort(key=lambda x: x[1], reverse=True)

        top_hits = [idx for idx, score in filtered_scores if score > 0.1][:3]

        if not top_hits:
            await message.answer("Извините, не смог найти ответ. Попробуйте переформулировать вопрос.")
            return

        response = ""
        for idx in top_hits:
            response += f"== {labels[idx]} ==\n{texts[idx][:500]}...\n\n"

        await message.answer(response)

    else:
        await message.answer("Напишите /start чтобы начать.")

def split_message(text, limit=4000):
    messages = []
    while len(text) > limit:
        split_pos = text.rfind('\n', 0, limit)
        if split_pos == -1:
            split_pos = limit
        messages.append(text[:split_pos])
        text = text[split_pos:].lstrip('\n')
    messages.append(text)
    return messages

async def main():
    nest_asyncio.apply()
    await dp.start_polling(bot)

if __name__ == "__main__":
    asyncio.run(main())

Received SIGINT signal
